<a href="https://colab.research.google.com/github/hgvo9/hw5-network/blob/main/Copy_of_proj4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sat Jun 11 10:59:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%shell

apt update
apt-get install g++ freeglut3-dev build-essential libx11-dev libxmu-dev libxi-dev libglu1-mesa libglu1-mesa-dev

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [806 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelea

In [ ]:
%%writefile cuda_ray.cu

#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

#define CUDA 0
#define OPENMP 1
#define SPHERES 20

#define rnd( x ) (x * rand() / RAND_MAX)
#define INF 2e10f
#define DIM 2048

struct Sphere {
    float   r,b,g;
    float   radius;
    float   x,y,z;
    __device__ float hit( float ox, float oy, float *n ) {
        float dx = ox - x;
        float dy = oy - y;
        if (dx*dx + dy*dy < radius*radius) {
            float dz = sqrtf( radius*radius - dx*dx - dy*dy );
            *n = dz / sqrtf( radius * radius );
            return dz + z;
        }
        return -INF;
    }
};

__global__ void kernel( Sphere *s, unsigned char *ptr ) {
   
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int offset = x + y * blockDim.x * gridDim.x;
    float   ox = (x - DIM/2);
    float   oy = (y - DIM/2);

    float   r=0, g=0, b=0;
    float   maxz = -INF;
    for(int i=0; i<SPHERES; i++) {
        float   n;
        float   t = s[i].hit( ox, oy, &n );
        if (t > maxz) {
            float fscale = n;
            r = s[i].r * fscale;
            g = s[i].g * fscale;
            b = s[i].b * fscale;
            maxz = t;
        }
    } 

    ptr[offset*4 + 0] = (int)(r * 255);
    ptr[offset*4 + 1] = (int)(g * 255);
    ptr[offset*4 + 2] = (int)(b * 255);
    ptr[offset*4 + 3] = 255;
}

void ppm_write(unsigned char* bitmap, int xdim,int ydim, FILE* fp)
{
	int i,x,y;
	fprintf(fp,"P3\n");
	fprintf(fp,"%d %d\n",xdim, ydim);
	fprintf(fp,"255\n");
	for (y=0;y<ydim;y++) {
		for (x=0;x<xdim;x++) {
			i=x+y*xdim;
			fprintf(fp,"%d %d %d ",bitmap[4*i],bitmap[4*i+1],bitmap[4*i+2]);
		}
		fprintf(fp,"\n");
	}
}

int main(int argc, char* argv[])
{
	unsigned char* bitmap;
	unsigned char *device_bitmap;

	srand(time(NULL));

	FILE* fp = fopen("result.ppm","w");

	Sphere *temp_s = (Sphere*)malloc( sizeof(Sphere) * SPHERES );
	Sphere *device_s;
	cudaMalloc((void**)&device_s, sizeof(Sphere) * SPHERES);
	for (int i=0; i<SPHERES; i++) {
		temp_s[i].r = rnd( 1.0f );
		temp_s[i].g = rnd( 1.0f );
		temp_s[i].b = rnd( 1.0f );
		temp_s[i].x = rnd( 2000.0f ) - 1000;
		temp_s[i].y = rnd( 2000.0f ) - 1000;
		temp_s[i].z = rnd( 2000.0f ) - 1000;
		temp_s[i].radius = rnd( 200.0f ) + 40;
	}
	cudaMemcpy(device_s, temp_s, sizeof(Sphere) * SPHERES, cudaMemcpyHostToDevice);
	bitmap=(unsigned char*)malloc(sizeof(unsigned char)*DIM*DIM*4);
	cudaMalloc((void**)&device_bitmap, sizeof(unsigned char)*DIM*DIM*4);
	dim3    grids(DIM/4,DIM/4);
  dim3    threads(4,4);
  kernel<<<grids,threads>>>( device_s, device_bitmap );
	cudaMemcpy(bitmap, device_bitmap, sizeof(unsigned char)*DIM*DIM*4, cudaMemcpyDeviceToHost);
	ppm_write(bitmap,DIM,DIM,fp);

	fclose(fp);
	free(bitmap);
	free(temp_s);
	cudaFree(device_bitmap);
	cudaFree(device_s);

	return 0;
}

Overwriting cuda_ray.cu


In [ ]:
%%shell

nvcc cuda_ray.cu
time ./a.out
nvprof ./a.out


real	0m0.918s
user	0m0.650s
sys	0m0.258s
==1888== NVPROF is profiling process 1888, command: ./a.out
==1888== Profiling application: ./a.out
==1888== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   76.35%  9.2660ms         1  9.2660ms  9.2660ms  9.2660ms  [CUDA memcpy DtoH]
                   23.64%  2.8685ms         1  2.8685ms  2.8685ms  2.8685ms  kernel(Sphere*, unsigned char*)
                    0.02%  1.8560us         1  1.8560us  1.8560us  1.8560us  [CUDA memcpy HtoD]
      API calls:   94.91%  277.32ms         2  138.66ms  119.68us  277.20ms  cudaMalloc
                    4.57%  13.362ms         2  6.6812ms  15.074us  13.347ms  cudaMemcpy
                    0.31%  909.14us         2  454.57us  222.06us  687.08us  cudaFree
                    0.13%  375.33us         1  375.33us  375.33us  375.33us  cuDeviceTotalMem
                    0.05%  149.52us       101  1.4800us     140ns  62.981us  cuDeviceGetAttr

In [ ]:
%%writefile thrust_ex.cu

#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/reduce.h>
#include <thrust/functional.h>
#include <thrust/random.h>
#include <cmath>

#define N 1000000

int main(void)
{
  thrust::host_vector<int> h_vec(N);

  thrust::counting_iterator<int>x(0);

  for(int i=0; i<N; ++i, ++x) {
      h_vec.push_back((4.0/(1+pow(*x, 2))) * (*x - *--x));
  }

  thrust::device_vector<int> d_vec = h_vec;

  thrust::plus<int> sumOperator;

  int sum = thrust::reduce(d_vec.begin(), d_vec.end(), 0, sumOperator);

  std::cout << sum << std::endl;

  return 0;
}

Overwriting thrust_ex.cu


In [ ]:
%%shell

nvcc thrust_ex.cu
./a.out

4000000
